This is the first of 3 notebooks that are used to process data and run SIR epidemic model to analyze impact of urban mobility due to public transport


In [1]:
import pandas as pd
import json
import itertools

#Get all bus routes and organize them in arrays
bus_routes = pd.read_csv('./data/bmtc_dump.csv')
route_nums = bus_routes['route_no']
all_bus_route_stops = bus_routes['map_json_content']
#print(all_bus_route_stops)
all_routes = []

for i in range(0, len(all_bus_route_stops)):
    routes_json_array = json.loads(all_bus_route_stops[i])
    route_num = route_nums[i]
    routes = []
    for item in routes_json_array:
        ll_array = item['latlons']
        x = route_num + "_" + ll_array[0] + "_" + ll_array[1]
        routes.append(x)

    all_routes.append(routes)

print(len(all_routes))
od_combinations = []
for route in all_routes:
    for subset in itertools.combinations(route, 2):
        od_combinations.append(subset)

df = pd.DataFrame(od_combinations)

#saving the dataframe
df.to_csv('od.csv')

print(df)

2045
                                               0  \
0            1_12.9197565816171_77.5923588994416   
1            1_12.9197565816171_77.5923588994416   
2            1_12.9197565816171_77.5923588994416   
3            1_12.9197565816171_77.5923588994416   
4            1_12.9197565816171_77.5923588994416   
...                                          ...   
1053481        600CF_12.9167048167_77.6067416836   
1053482        600CF_12.9167048167_77.6067416836   
1053483  600CF_12.9166423284103_77.6090389222481   
1053484  600CF_12.9166423284103_77.6090389222481   
1053485         600CF_12.9163629189_77.615990527   

                                           1  
0        1_12.9226893214618_77.5933845451659  
1        1_12.9231859777075_77.5887716805949  
2        1_12.9279596029171_77.5876041867249  
3          1_12.9284826437007_77.58420959115  
4              1_12.9317783285_77.5839063618  
...                                      ...  
1053481     600CF_12.9163629189_77.615990

In this cell we locate the bus stops for every single ward 

In [ ]:
import pandas as pd
import json
import itertools
import numpy as np
from shapely.geometry.polygon import Point
from shapely.geometry.polygon import Polygon

#Get all ods
ods = pd.read_csv('./data/od.csv')
origins = ods['origin']
destinations = ods['destination']

f = open('./data/wards.json',)

wards_json = json.load(f)
wards = wards_json['features']
ward_polygons = {}
polygon = None
lats_vect = None
lons_vect = None
for ward in wards:
    ward_properties = ward['properties']
    ward_geo = ward['geometry']
    polygon_type = ward_geo['type']
    if polygon_type == "Polygon":
        lon_lats_vect = ward_geo['coordinates']
        lons_lats = lon_lats_vect[0]
        lons = []
        lats = []
        for i in range(0, len(lons_lats)):
            lon_lat = lons_lats[i]
            lons.append(lon_lat[0])
            lats.append(lon_lat[1])

        lats_vect = np.array(lats)
        lons_vect = np.array(lons)
        lons_lats_vect = np.column_stack((lons_vect, lats_vect))
        polygon = Polygon(lons_lats_vect)
        ward_polygons.update({ward_properties['WARD_NO']: polygon})
    elif polygon_type == "MultiPolygon":
        lons_lats_vects = ward_geo['coordinates']
        n = 1
        for lons_lats_vect in lons_lats_vects:
            lon_lats = lons_lats_vect[0]
            lons = []
            lats = []
            for i in range(0, len(lon_lats)):
                lon_lat = lon_lats[i]
                lons.append(lon_lat[0])
                lats.append(lon_lat[1])

            lats_vect = np.array(lats)
            lons_vect = np.array(lons)
            lons_lats_vect = np.column_stack((lons_vect, lats_vect))
            polygon = Polygon(lons_lats_vect)
            ward_polygons.update({(ward_properties['WARD_NO'] + "_" + str(n)) : polygon})
            n = n + 1
print(len(ward_polygons))
df_rows = []
m = 0

#od_summary = pd.DataFrame(columns=['route_no','origin_lat','origin_lon','destination_lat','destination_lon','origin_ward','destination_ward'])
for m in range(0, len(origins)):
    origin_ward = None
    origin_lat = None
    origin_lon = None
    destination_lat = None
    destination_lon = None
    origin_ward = None
    destination_ward = None

    for ward_no, ward_poly in ward_polygons.items():
        origin_lat_lon = (origins[m]).split("_")
        route_num = origin_lat_lon[0]
        origin_lat = origin_lat_lon[1]
        origin_lon = origin_lat_lon[2]
        origin_point = Point(float(origin_lon), float(origin_lat))

        if ward_poly.contains(origin_point):
            origin_ward = ward_no
            break

        if origin_point.within(ward_poly):
            origin_ward = ward_no
            break

    for ward_no, ward_poly in ward_polygons.items():
        destination_lat_lon = (destinations[m]).split("_")
        route_num = destination_lat_lon[0]
        destination_lat = destination_lat_lon[1]
        destination_lon = destination_lat_lon[2]
        destination_point = Point(float(destination_lon), float(destination_lat))

        if ward_poly.contains(destination_point):
            destination_ward = ward_no
            break

        if destination_point.within(ward_poly):
            destination_ward = wardd_no
            break

    row = []
    row.append(route_num)
    row.append(origin_lat)
    row.append(origin_lon)
    row.append(destination_lat)
    row.append(destination_lon)
    row.append(origin_ward)
    row.append(destination_ward)

    df_rows.append(row)

    print(m)
    m = m + 1

od_summary1 = pd.DataFrame(df_rows)

od_summary1.to_csv('od_summary.csv')
